In [2]:
from pathlib import Path
import pandas as pd


def convert_csv_datetime(csv_path: Path, save_path: Path):
    """Reads a CSV file, converts 'start', 'end', and 'single_marker' columns to datetime, and saves back."""
    df = pd.read_csv(csv_path)
    for col in ['start', 'end', 'single_marker']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
    df.to_csv(save_path, index=False)


convert_csv_datetime(
    Path('/Users/julian/Developer/SeizurePredictionThesis/test_files/annotations_to_combine/start_marker_end.csv'),
    Path(
        '/Users/julian/Developer/SeizurePredictionThesis/test_files/annotations_to_combine/start_marker_end_datetime.csv')
)